In [1]:
import re
import librosa, librosa.display
import numpy as np
import scipy, matplotlib.pyplot as plt, IPython.display as ipd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import credentials as cd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from itertools import chain

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))   # resizes jupyter notebook 

pd.set_option('display.max_columns', None)                             # max number of rows table shows

# Retrieve 600k song dataset and do data exploration

In [3]:
spotify_df = pd.read_csv("tracks.csv")
spotify_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [4]:
artist_df = pd.read_csv("artists.csv")
artist_df

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0
...,...,...,...,...,...
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,['black comedy'],Ali Siddiq,34
1162091,6LogY6VMM3jgAE6fPzXeMl,46.0,[],Rodney Laney,2
1162092,19boQkDEIay9GaVAWkUhTa,257.0,[],Blake Wexler,10
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15


In [5]:
artist_df['genres_upd'] = artist_df['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [6]:
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [7]:
spotify_df['artists'].values[0]

"['Uli']"

In [8]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,[Uli]
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,[Fernando Pessoa]
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,[Ignacio Corsini]
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,[Ignacio Corsini]
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,[Dick Haymes]


In [9]:
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df[spotify_df['artists_upd_v1'].apply(lambda x: not x)]
spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists_upd_v1'] )

# Some song names are literal numbers (eg. 123, 2002) so cast the whole column as a string
spotify_df['name'] = spotify_df['name'].map(str)

spotify_df['artists_song'] = spotify_df.apply(lambda row: row['artists_upd'][0]+row['name'],axis = 1)
spotify_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)
spotify_df.drop_duplicates('artists_song',inplace = True)
spotify_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
538433,3u1C6nWVRoP5F0w8gGrDL3,사랑의 미로,25,222380,0,['최진희'],['1NSrAf8XJYJVgAXKoxaMet'],1987-06-01,0.367,0.194,7,-19.057,1,0.0400,0.617,0.000006,0.1620,0.3670,144.316,4,[최진희],[],[최진희],최진희사랑의 미로
404349,1Mv4u308L16NZDZiD6HZCy,사랑은 힘든가봐,28,213440,0,['지수'],['4c9QIMfEbIIynuaswyxGx9'],2005-12-23,0.675,0.785,4,-5.026,0,0.0280,0.379,0.000000,0.3530,0.6230,103.008,4,[지수],[],[지수],지수사랑은 힘든가봐
210091,1jvoY322nxyKXq8OBhgmSY,어떡하죠,44,244360,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2011-10-13,0.606,0.341,0,-7.094,1,0.0513,0.779,0.000000,0.1440,0.2940,135.667,4,[지선],[],[지선],지선어떡하죠
270610,2ghebdwe2pNXT4eL34T7pW,그아픔까지사랑한거야,32,237688,0,['조정현'],['2WTpsPucygbYRnCnoEUkJQ'],1989-06-15,0.447,0.215,10,-16.478,1,0.0272,0.568,0.000001,0.0649,0.1770,71.979,4,[조정현],[],[조정현],조정현그아픔까지사랑한거야
208974,7rxpWwcXNgDUXl0wN0gUvp,천국의 기억 장정우 Version,31,280372,0,['장정우'],['5L7zKs2ftwENWOMI7LFaN1'],2003-12-24,0.494,0.656,7,-6.347,0,0.0262,0.659,0.000007,0.1110,0.4200,82.003,4,[장정우],[],[장정우],장정우천국의 기억 장정우 Version
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158908,3YCC4oCzagcfx4s3fp3XFz,Bandolera,35,322926,1,"[""At' Fat"", 'Yemil', 'Boza', 'El Tachi']","['455JBMYIT1lWf1Djj4rDc7', '7g4cPtKxNx146qdaY9...",2020-11-17,0.502,0.814,5,-2.688,0,0.1330,0.524,0.000000,0.1510,0.6720,72.344,4,"[ Fat"", , , , , ]",[At' Fat],"[ Fat"", , , , , ]","Fat"", Bandolera"
537682,5sr38tHMSPpaENiNcjUpFi,Cathrina,10,184560,0,"[""Henry D' Cruz"", ""Helen D'Souza""]","['3nH2WLSi1MhvlmJNkbLbIv', '4NR8lI9YoPWo5v2Kv6...",1976-01-01,0.661,0.677,1,-8.447,0,0.1160,0.683,0.001540,0.0361,0.8090,116.855,4,"[ Cruz"", ""Helen D]","[Henry D' Cruz, Helen D'Souza]","[ Cruz"", ""Helen D]","Cruz"", ""Helen DCathrina"
281020,6Pu2leLWEfThVIqJilw6O9,The Passion of Our Lord according to St. Matth...,4,79027,0,"[""Boys' Choir of The Church of The Transfigura...","['47EwjzhfZKotjVtvlQFNFS', '5H155SiWINLkYRElqw...",1963,0.177,0.208,11,-22.988,0,0.0388,0.988,0.946000,0.0982,0.0587,132.053,3,"[ Choir of The Church of The Transfiguration"",...",[Boys' Choir of The Church of The Transfigurat...,"[ Choir of The Church of The Transfiguration"",...","Choir of The Church of The Transfiguration"", ..."
294380,7f092ECp06XbFS6Ms5Yk6R,Mia Oraia Petalouda,26,100313,0,"[""Typaldos' Children's Choire""]",['7JJrBciIAQBPOBB5U6u49p'],2014-06-08,0.773,0.183,0,-9.370,1,0.0370,0.956,0.000000,0.1190,0.8640,79.950,4,[ Children],[Typaldos' Children's Choire],[ Children],ChildrenMia Oraia Petalouda


In [10]:
artist_df

,id,followers,genres,name,popularity,genres_upd
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0,[]
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0,[]
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0,[]
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0,[]
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0,[]
...,...,...,...,...,...,...
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,['black comedy'],Ali Siddiq,34,[black_comedy]
1162091,6LogY6VMM3jgAE6fPzXeMl,46.0,[],Rodney Laney,2,[]
1162092,19boQkDEIay9GaVAWkUhTa,257.0,[],Blake Wexler,10,[]
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15,[black_comedy]


In [11]:
artists_exploded = spotify_df[['artists_upd','id_artists']].explode('artists_upd')
artists_exploded_enriched = artists_exploded.merge(artist_df, how = 'left', left_on = 'artists_upd',right_on = 'name')
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres.isnull()]
artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull['id_artists'] =='6KuQTIu1KoTTkLXKrwlLPV']

artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id_artists')['genres'].apply(list).reset_index()
artists_genres_consolidated['consolidated_genre_lists'] = artists_genres_consolidated['genres'].apply(lambda x: x[0])
artists_genres_consolidated.dtypes

id_artists                  object
genres                      object
consolidated_genre_lists    object
dtype: object

In [12]:
spotify_df = spotify_df.merge(artists_genres_consolidated[['id_artists','consolidated_genre_lists']], on = 'id_artists',how = 'left')
spotify_df[spotify_df['name'] == 'Titanium']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidated_genre_lists
229086,3Lp3nA3GZ9kgLg3yoqJ4NX,Titanium,64,210008,0,['Madilyn Bailey'],['0NxhAEPOSeCg6vypFr7yjU'],2012-05-06,0.501,0.219,3,-8.226,1,0.0357,0.8120,0.000000,0.1250,0.264,120.931,4,[Madilyn Bailey],[],[Madilyn Bailey],Madilyn BaileyTitanium,['viral pop']
336630,7dpNv1EcetEjnD1bo3Rm8d,Titanium,16,231735,0,"['Halldór Gunnar', 'Sverrir Bergmann']","['15mE2JovaBOHOVgd2rkLC2', '6vZQqLEETz3r5dQFKn...",2012-01-01,0.656,0.162,6,-9.023,0,0.0398,0.7300,0.000001,0.1110,0.198,119.580,4,"[Halldór Gunnar, Sverrir Bergmann]",[],"[Halldór Gunnar, Sverrir Bergmann]",Halldór GunnarTitanium,['icelandic pop']
421158,2GusQcB3m4nkLJRWAfcDNB,Titanium,47,246187,0,"['David Guetta', 'Sia']","['1Cs0zKBU1kc0i8ypK3B9ai', '5WUlDfRSoLAfcVSX1W...",2013-01-01,0.598,0.613,0,-8.942,0,0.1040,0.0755,0.057100,0.1540,0.301,126.034,4,"[David Guetta, Sia]",[],"[David Guetta, Sia]",David GuettaTitanium,"['big room', 'dance pop', 'edm', 'pop', 'pop d..."
422009,2qzW5AK8ldKUOWUETwAkZY,Titanium,69,127750,1,['Dave'],['6Ip8FS7vWT1uKkJSweANQK'],2021-04-10,0.959,0.394,0,-8.687,0,0.4370,0.1520,0.000001,0.1050,0.510,121.008,4,[Dave],[],[Dave],DaveTitanium,[]
446820,28bD8nZOrb9RsHK11INABa,Titanium,1,247766,0,['Celebration Collective'],['3FpYem9YTIOUOiu2u7cPWj'],2014-10-24,0.701,0.571,0,-7.298,0,0.0479,0.0570,0.001180,0.6250,0.212,125.973,4,[Celebration Collective],[],[Celebration Collective],Celebration CollectiveTitanium,NaN
460829,0knwYd0kPonZMwaNWLLudK,Titanium,59,227563,0,['Boyce Avenue'],['7CQwac16i1W5ej8YpuL3dv'],2015-04-28,0.804,0.547,4,-7.769,0,0.0665,0.1940,0.000000,0.0888,0.507,126.013,4,[Boyce Avenue],[],[Boyce Avenue],Boyce AvenueTitanium,"['neo mellow', 'pop rock', 'viral pop']"


# Application of OHE and TF-IDF

In [13]:
spotify_df['year'] = spotify_df['release_date'].apply(lambda x: x.split('-')[0])
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values
ohe_cols = 'popularity'

# create 5 point buckets for popularity 
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

# tfidf can't handle nulls so fill any null values with an empty list
spotify_df = spotify_df.dropna(subset=['consolidated_genre_lists'])
spotify_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidated_genre_lists,year,popularity_red
0,3u1C6nWVRoP5F0w8gGrDL3,사랑의 미로,25,222380,0,['최진희'],['1NSrAf8XJYJVgAXKoxaMet'],1987-06-01,0.367,0.194,7,-19.057,1,0.0400,0.6170,0.000006,0.1620,0.367,144.316,4,[최진희],[],[최진희],최진희사랑의 미로,['trot'],1987,5
2,1jvoY322nxyKXq8OBhgmSY,어떡하죠,44,244360,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2011-10-13,0.606,0.341,0,-7.094,1,0.0513,0.7790,0.000000,0.1440,0.294,135.667,4,[지선],[],[지선],지선어떡하죠,[],2011,8
4,7rxpWwcXNgDUXl0wN0gUvp,천국의 기억 장정우 Version,31,280372,0,['장정우'],['5L7zKs2ftwENWOMI7LFaN1'],2003-12-24,0.494,0.656,7,-6.347,0,0.0262,0.6590,0.000007,0.1110,0.420,82.003,4,[장정우],[],[장정우],장정우천국의 기억 장정우 Version,[],2003,6
5,0cEvzbXjxkOxgBUmBUcHZW,그것만은..,32,294452,0,['장정우'],['5L7zKs2ftwENWOMI7LFaN1'],2003-12-24,0.311,0.487,4,-6.847,0,0.0265,0.4840,0.000000,0.2410,0.159,59.464,4,[장정우],[],[장정우],장정우그것만은..,[],2003,6
8,6uRXQW8BqB3N9WKfe9gfdw,Title 허밍,37,102307,0,['이경섭'],['191huMISbbIeUELiiEGZ7L'],2007-11-09,0.825,0.604,7,-6.348,0,0.0417,0.3760,0.000001,0.0875,0.898,98.272,4,[이경섭],[],[이경섭],이경섭Title 허밍,[],2007,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523385,5UbK7BJAIhkUIYYLUABLdN,The Last Supper,30,428507,0,"['""Jesus Christ Superstar"" Apostles - Original...","['1T1Hnf7jWiJPFUTi6voWwU', '479Yp6DvyXoIaCssAx...",1970-01-01,0.397,0.396,7,-8.094,1,0.0345,0.5930,0.000000,0.0997,0.232,121.013,4,"[""Jesus Christ Superstar"" Apostles - Original ...",[Jesus Christ Superstar],"[""Jesus Christ Superstar"" Apostles - Original ...","""Jesus Christ Superstar"" Apostles - Original S...",[],1970,6
523386,2itG1gLI0n9tH1DZKJbPd2,The Warrior,11,249493,0,"['""Ipi Ntombi"" 1975 Original Cast']",['2rV4kKW3Yvqh40hpDH1gpy'],1975-01-01,0.749,0.590,0,-5.699,1,0.0528,0.1580,0.003260,0.1440,0.574,116.904,4,"[""Ipi Ntombi"" 1975 Original Cast]",[Ipi Ntombi],"[""Ipi Ntombi"" 1975 Original Cast]","""Ipi Ntombi"" 1975 Original CastThe Warrior",['xhosa'],1975,2
523387,52V0laIHD3rvFZrwqDPHFe,Shosholoza,17,231733,0,"['""Ipi Ntombi"" 1975 Original Cast']",['2rV4kKW3Yvqh40hpDH1gpy'],1975-01-01,0.721,0.534,0,-10.546,1,0.0506,0.4960,0.000000,0.1110,0.923,114.075,4,"[""Ipi Ntombi"" 1975 Original Cast]",[Ipi Ntombi],"[""Ipi Ntombi"" 1975 Original Cast]","""Ipi Ntombi"" 1975 Original CastShosholoza",['xhosa'],1975,3
523388,5qhaazvxbhR8mBlo1LXTuY,Narration: Mama Tembu's Wedding,7,182733,0,"['""Ipi Ntombi"" 1975 Original Cast']",['2rV4kKW3Yvqh40hpDH1gpy'],1975-01-01,0.604,0.803,0,-6.785,1,0.4300,0.4260,0.000000,0.2480,0.916,134.034,4,"[""Ipi Ntombi"" 1975 Original Cast]",[Ipi Ntombi],"[""Ipi Ntombi"" 1975 Original Cast]","""Ipi Ntombi"" 1975 Original CastNarration: Mama...",['xhosa'],1975,1


In [14]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df


In [19]:
#function to build entire feature set
def create_feature_set(df, float_cols):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    
    #tfidf genre lists
    tfidf = TfidfVectorizer(stop_words=None)
    tfidf_matrix =  tfidf.fit_transform(df['consolidated_genre_lists'])
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop = True, inplace=True)

    #explicity_ohe = ohe_prep(df, 'explicit','exp')    
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.5

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    #add song id
    final['id']=df['id'].values
    
    return final

In [20]:
spotify_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,consolidated_genre_lists,year,popularity_red
1490,6qWEpnyU43Bcrx3vfbGuVm,Touch You,77,206180,0,"['遠野高志', '加島優', '矢口恭介', 'ジミー', '百合絢斗', '田村唯', ...","['3dkW1L7GIhispeiHo9uzM8', '3ynBix87InXjEn5L9L...",2018-08-15,0.603,0.964,8,-2.886,0,0.0487,0.00655,0.000003,0.1430,0.7960,135.028,4,"[遠野高志, 加島優, 矢口恭介, ジミー, 百合絢斗, 田村唯, 明美圭一, 糸目幸士郎,...",[],"[遠野高志, 加島優, 矢口恭介, ジミー, 百合絢斗, 田村唯, 明美圭一, 糸目幸士郎,...",遠野高志Touch You,['yaoi'],2018,15
6255,7dH0dpi751EoguDDg3xx6J,ドライフラワー,81,285587,0,['優里'],['0ixzjrK1wkN2zWBXt3VW3W'],2020-10-25,0.463,0.624,7,-5.118,1,0.0274,0.42600,0.000000,0.2210,0.5210,147.929,4,[優里],[],[優里],優里ドライフラワー,"['j-pop', 'japanese singer-songwriter', 'japan...",2020,16
13981,2iJuuzV8P9Yz0VSurttIV5,Scream & Shout,80,283400,1,"['will.i.am', 'Britney Spears']","['085pc2PYOi8bGKj0PNjekA', '26dSoYclwsYLMAKD3t...",2013-01-01,0.772,0.685,5,-6.849,1,0.0696,0.01900,0.000090,0.1310,0.5010,130.033,4,"[will.i.am, Britney Spears]",[],"[will.i.am, Britney Spears]",will.i.amScream & Shout,"['dance pop', 'pop', 'pop rap', 'post-teen pop']",2013,16
14054,51Of5p3lKZeOg6itfs4og4,Lovefool,83,190488,0,['twocolors'],['7ACEUD7UsmmXrnj4OLt8f9'],2020-05-08,0.677,0.766,6,-6.896,1,0.0568,0.02190,0.000007,0.1290,0.1980,123.062,4,[twocolors],[],[twocolors],twocolorsLovefool,"['edm', 'pop edm', 'tropical house']",2020,16
14197,4DnHEa1fuf3bn61AwT8Qiv,Ordentlich,78,168004,1,['t-low'],['3tQzzidoPfVifoURnDfgmD'],2020-09-25,0.832,0.609,8,-7.474,1,0.2880,0.13900,0.000000,0.0710,0.6410,160.025,4,[t-low],[],[t-low],t-lowOrdentlich,['german trap'],2020,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523284,0kEZlJh4mK1QRfb3CT5LPk,Kill Yourself (Part III),79,145078,1,['$uicideboy$'],['1VPmR4DJC1PlOtd0IADAO0'],2015-09-21,0.753,0.680,5,-7.092,0,0.0349,0.32600,0.000041,0.4230,0.1920,105.977,4,[$uicideboy$],[],[$uicideboy$],$uicideboy$Kill Yourself (Part III),"['dark trap', 'new orleans rap', 'underground ...",2015,15
523297,30QR0ndUdiiMQMA9g1PGCm,"...And to Those I Love, Thanks for Sticking Ar...",80,168490,1,['$uicideboy$'],['1VPmR4DJC1PlOtd0IADAO0'],2020-02-14,0.792,0.511,2,-6.876,1,0.0409,0.12400,0.000090,0.1400,0.1110,113.983,4,[$uicideboy$],[],[$uicideboy$],"$uicideboy$...And to Those I Love, Thanks for ...","['dark trap', 'new orleans rap', 'underground ...",2020,16
523326,3oGbHF3Kdwf3AsRCbBjUxu,Moon & Stars (feat. Maggie Lindemann),76,187587,1,"['$NOT', 'Maggie Lindemann']","['5IbEL2xjRtKsunfmsahLuO', '0uGk2czvcpWQA383Im...",2020-03-06,0.713,0.470,9,-8.698,1,0.0864,0.60600,0.000013,0.3060,0.0825,84.518,4,"[$NOT, Maggie Lindemann]",[],"[$NOT, Maggie Lindemann]",$NOTMoon & Stars (feat. Maggie Lindemann),"['dark trap', 'florida rap', 'underground hip ...",2020,15
523327,3PXi72ZtSqx1PZc40KS0Qj,Mean,76,125853,1,"['$NOT', 'Flo Milli']","['5IbEL2xjRtKsunfmsahLuO', '08PvCOlef4xdOr20jF...",2020-10-30,0.844,0.720,1,-7.418,0,0.3000,0.16900,0.000000,0.0797,0.6310,99.917,4,"[$NOT, Flo Milli]",[],"[$NOT, Flo Milli]",$NOTMean,"['dark trap', 'florida rap', 'underground hip ...",2020,15


In [21]:
spotify_df = spotify_df[spotify_df['popularity'] > 75]
complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols)#.mean(axis = 0)
complete_feature_set.to_csv("test.csv")
# complete_feature_set.rename(columns={'spotify_id': 'id'}, inplace=True)
complete_feature_set

,genre|acoustic,genre|adult,genre|afrofuturism,genre|afropop,genre|afroswing,genre|alabama,genre|album,genre|alt,genre|alternative,genre|ambient,genre|and,genre|anime,genre|anthem,genre|antigas,genre|arena,genre|argentine,genre|argentino,genre|art,genre|athens,genre|atl,genre|austindie,genre|australian,genre|azontobeats,genre|bachata,genre|background,genre|baiano,genre|baixada,genre|band,genre|banda,genre|barbadian,genre|baroque,genre|bass,genre|basshall,genre|baton,genre|beatlesque,genre|beats,genre|bedroom,genre|belarusian,genre|belgian,genre|bh,genre|big,genre|birmingham,genre|blues,genre|bmore,genre|bollywood,genre|boricua,genre|boston,genre|bounce,genre|boy,genre|brasileira,genre|brasileiro,genre|brazilian,genre|brill,genre|british,genre|britpop,genre|broadway,genre|brooklyn,genre|brostep,genre|bubblegrunge,genre|bubblegum,genre|building,genre|cafe,genre|cali,genre|canadian,genre|candy,genre|carioca,genre|carolina,genre|ccm,genre|celtic,genre|champeta,genre|channel,genre|chicago,genre|chicano,genre|chihuahua,genre|chileno,genre|chill,genre|chillhop,genre|chillwave,genre|christian,genre|circuit,genre|classic,genre|cloud,genre|coast,genre|colombian,genre|colombiana,genre|colombiano,genre|conscient,genre|consciente,genre|conscious,genre|contemporary,genre|corrido,genre|corridos,genre|country,genre|cristiano,genre|cubaton,genre|cumbia,genre|czech,genre|czsk,genre|dance,genre|dancehall,genre|dark,genre|das,genre|dawn,genre|de,genre|deep,genre|dembow,genre|desi,genre|detroit,genre|df,genre|dfw,genre|diego,genre|dirty,genre|disco,genre|dmv,genre|dominicana,genre|dominicano,genre|double,genre|dream,genre|drill,genre|drumming,genre|drums,genre|dubstep,genre|dutch,genre|east,genre|edm,genre|edmonton,genre|el,genre|electro,genre|electronic,genre|electropop,genre|emo,genre|en,genre|escape,genre|espanol,genre|etherpop,genre|euro,genre|eurodance,genre|europop,genre|experimental,genre|favela,genre|fi,genre|filmi,genre|filter,genre|flamenco,genre|florida,genre|flow,genre|fluminense,genre|folk,genre|forro,genre|francais,genre|francoton,genre|french,genre|funk,genre|funky,genre|fusion,genre|game,genre|gaming,genre|gangster,genre|garage,genre|gauze,genre|german,genre|girl,genre|glam,genre|gold,genre|greek,genre|groove,genre|group,genre|grunge,genre|guaracha,genre|hamburg,genre|hard,genre|hardcore,genre|hawaiian,genre|heartland,genre|hi,genre|hip,genre|holler,genre|hollywood,genre|hop,genre|house,genre|houston,genre|icelandic,genre|idol,genre|indie,genre|indiecoustica,genre|indietronica,genre|indonesian,genre|international,genre|invasion,genre|irish,genre|island,genre|islands,genre|italian,genre|italiano,genre|italo,genre|japanese,genre|jersey,genre|kentucky,genre|kombina,genre|la,genre|latin,genre|latina,genre|latino,genre|lgbtq,genre|lilith,genre|lo,genre|lovers,genre|madchester,genre|mariachi,genre|marseille,genre|media,genre|melancholia,genre|melbourne,genre|mellow,genre|melodic,genre|meme,genre|merseybeat,genre|metal,genre|metalcore,genre|metropopolis,genre|mexican,genre|mexicano,genre|miami,genre|milan,genre|minneapolis,genre|minnesota,genre|mississippi,genre|mn,genre|modern,genre|moombahton,genre|motown,genre|movie,genre|mpb,genre|music,genre|musica,genre|nacional,genre|neo,genre|neon,genre|new,genre|newcastle,genre|nigerian,genre|ninja,genre|noise,genre|norteno,genre|north,genre|norwegian,genre|nova,genre|novo,genre|nrg,genre|nsw,genre|nu,genre|nuevo,genre|nyc,genre|nz,genre|oakland,genre|ohio,genre|oklahoma,genre|old,genre|orleans,genre|ostentacao,genre|otacore,genre|outlaw,genre|oxford,genre|pagode,genre|panamanian,genre|paso,genre|paulista,genre|permanent,genre|peruvian,genre|philly,genre|piano,genre|piseiro,genre|pittsburgh,genre|pixie,genre|png,genre|polish,genre|political,genre|polynesian,genre|pop,genre|poprock,genre|poptimism,genre|popular,genre|portland,genre|portuguese,genre|post,genre|power,genre|progressive,genre|psych,genre|psychedelic,genre|puerto,genre|punjabi,genre|punk,genre|queen,genre|queens,genre|queer,genre|quiet

# Get the saved playlist ready

In [22]:
#Authen with Spotify

cid = '...'    # test with your own --> extract from spotify developer
secret = '...'
redirect_uri = "http://localhost:8881/"
scope = "user-follow-read user-read-recently-played user-library-read user-top-read user-read-currently-playing user-library-modify playlist-modify-private playlist-modify-public"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = cid, client_secret = secret, redirect_uri = redirect_uri,scope=scope))

In [23]:
#Retrieve the most recently saved songs
userid = sp.current_user()['id']

#Retrieve 50 most recently saved songs
results_saved = sp.current_user_saved_tracks(limit=50)

#Retrieve a playlist
saved_song = []
for idx, item in enumerate(results_saved['items']):
    added_time = item['added_at'].replace("T", " ").replace("Z", "")
    month_added = relativedelta(datetime.now(), datetime.strptime(added_time, '%Y-%m-%d %H:%M:%S')).months
    year = int(item['track']['album']['release_date'][:4])
    name = item['track']['name'] 
    
    artist = item['track']['artists'][0]['name'] 
    artist_info = sp.artist(item['track']["artists"][0]["external_urls"]["spotify"])
    popularity = artist_info['popularity']
    genres = artist_info["genres"]
    

    album = item['track']['album']['name'] 
    track_id = item['track']['id']
    saved_song.append([track_id, name, artist, album, month_added, added_time, genres, year, popularity])
    
saved_df = pd.DataFrame(data=saved_song, columns=['id', 'name', 'artist', 'album', 'month_added','date_added', 'genres', 'year', 'popularity'])
saved_df.set_index('id')
saved_df

,id,name,artist,album,month_added,date_added,genres,year,popularity
0,4eJ2IvNMxYUvOHkZBj260r,My King Forever - Live,Bethel Music,Homecoming (Live),0,2023-03-26 14:50:27,"[ambient worship, ccm, christian music, deep c...",2021,72
1,4aeyKGt2YHNnEtvVs0pmLQ,谁会被吻,Lil Jet,谁会被吻,3,2022-12-31 16:36:37,[chinese hip hop],2022,32
2,4qu63nuBpdn0qHUHuObEj1,Leave Before You Love Me (with Jonas Brothers),Marshmello,Leave Before You Love Me,3,2022-12-15 17:46:11,"[brostep, dance pop, edm, pop, pop dance, prog...",2021,85
3,6WEBwvsmpaoGxka0tSh5a7,Won't Go Home Without You,Maroon 5,It Won't Be Soon Before Long (International Ve...,4,2022-11-29 09:55:46,[pop],2007,87
4,0xHdrRs3i00vBbXzx0Hq67,This Is How I Thank The Lord,Mosaic MSC,This Is How I Thank The Lord,5,2022-10-20 05:31:43,"[anthem worship, ccm, christian music, world w...",2022,58
5,6I3mqTwhRpn34SLVafSH7G,Ghost,Justin Bieber,Justice,6,2022-09-28 14:53:50,"[canadian pop, pop]",2021,91
6,0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),Post Malone,beerbongs & bentleys,6,2022-09-24 07:08:38,"[dfw rap, melodic rap, rap]",2018,89
7,57F80059mWaWFvWG3tBTbO,The Edge Of Glory,Lady Gaga,Born This Way (International Special Edition V...,6,2022-09-24 07:08:20,"[art pop, dance pop, pop]",2011,87
8,1xz9uWsud2O07LXUcxN7vk,Crack the Tomb,Hulvey,COMA,6,2022-09-24 07:08:15,"[christian hip hop, christian trap]",2021,63
9,1jDJFeK9x3OZboIAHsY9k2,I'm Still Standing,Elton John,Too Low For Zero,6,2022-09-24 07:05:56,"[glam rock, mellow gold, piano rock]",1983,85


In [24]:
# Obtain song feature using Spotify API 

song_charact = pd.DataFrame()
for id in saved_df['id']:
    info = pd.DataFrame(sp.audio_features(id), index=[id])
    song_charact = pd.concat([info, song_charact])

user_df = pd.merge(saved_df, song_charact,  how='inner')
user_df

,id,name,artist,album,month_added,date_added,genres,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,4eJ2IvNMxYUvOHkZBj260r,My King Forever - Live,Bethel Music,Homecoming (Live),0,2023-03-26 14:50:27,"[ambient worship, ccm, christian music, deep c...",2021,72,0.414,0.555,9,-8.017,1,0.0272,0.055500,0.000015,0.0755,0.161,144.078,audio_features,spotify:track:4eJ2IvNMxYUvOHkZBj260r,https://api.spotify.com/v1/tracks/4eJ2IvNMxYUv...,https://api.spotify.com/v1/audio-analysis/4eJ2...,315767,3
1,4aeyKGt2YHNnEtvVs0pmLQ,谁会被吻,Lil Jet,谁会被吻,3,2022-12-31 16:36:37,[chinese hip hop],2022,32,0.704,0.585,0,-8.221,1,0.0424,0.416000,0.000000,0.1250,0.706,119.958,audio_features,spotify:track:4aeyKGt2YHNnEtvVs0pmLQ,https://api.spotify.com/v1/tracks/4aeyKGt2YHNn...,https://api.spotify.com/v1/audio-analysis/4aey...,181255,4
2,4qu63nuBpdn0qHUHuObEj1,Leave Before You Love Me (with Jonas Brothers),Marshmello,Leave Before You Love Me,3,2022-12-15 17:46:11,"[brostep, dance pop, edm, pop, pop dance, prog...",2021,85,0.721,0.738,7,-4.770,1,0.0403,0.002260,0.000004,0.1180,0.637,119.976,audio_features,spotify:track:4qu63nuBpdn0qHUHuObEj1,https://api.spotify.com/v1/tracks/4qu63nuBpdn0...,https://api.spotify.com/v1/audio-analysis/4qu6...,154983,4
3,6WEBwvsmpaoGxka0tSh5a7,Won't Go Home Without You,Maroon 5,It Won't Be Soon Before Long (International Ve...,4,2022-11-29 09:55:46,[pop],2007,87,0.737,0.615,3,-3.760,1,0.0317,0.014900,0.000000,0.1020,0.410,110.023,audio_features,spotify:track:6WEBwvsmpaoGxka0tSh5a7,https://api.spotify.com/v1/tracks/6WEBwvsmpaoG...,https://api.spotify.com/v1/audio-analysis/6WEB...,231173,4
4,0xHdrRs3i00vBbXzx0Hq67,This Is How I Thank The Lord,Mosaic MSC,This Is How I Thank The Lord,5,2022-10-20 05:31:43,"[anthem worship, ccm, christian music, world w...",2022,58,0.333,0.254,4,-9.369,1,0.0549,0.825000,0.000000,0.0795,0.272,189.168,audio_features,spotify:track:0xHdrRs3i00vBbXzx0Hq67,https://api.spotify.com/v1/tracks/0xHdrRs3i00v...,https://api.spotify.com/v1/audio-analysis/0xHd...,189933,3
5,6I3mqTwhRpn34SLVafSH7G,Ghost,Justin Bieber,Justice,6,2022-09-28 14:53:50,"[canadian pop, pop]",2021,91,0.601,0.741,2,-5.569,1,0.0478,0.185000,0.000029,0.4150,0.441,153.960,audio_features,spotify:track:6I3mqTwhRpn34SLVafSH7G,https://api.spotify.com/v1/tracks/6I3mqTwhRpn3...,https://api.spotify.com/v1/audio-analysis/6I3m...,153190,4
6,0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),Post Malone,beerbongs & bentleys,6,2022-09-24 07:08:38,"[dfw rap, melodic rap, rap]",2018,89,0.585,0.520,5,-6.136,0,0.0712,0.124000,0.000070,0.1310,0.129,159.801,audio_features,spotify:track:0e7ipj03S05BNilyu5bRzt,https://api.spotify.com/v1/tracks/0e7ipj03S05B...,https://api.spotify.com/v1/audio-analysis/0e7i...,218147,4
7,57F80059mWaWFvWG3tBTbO,The Edge Of Glory,Lady Gaga,Born This Way (International Special Edition V...,6,2022-09-24 07:08:20,"[art pop, dance pop, pop]",2011,87,0.583,0.768,9,-6.477,1,0.0410,0.000323,0.016200,0.1090,0.357,127.952,audio_features,spotify:track:57F80059mWaWFvWG3tBTbO,https://api.spotify.com/v1/tracks/57F80059mWaW...,https://api.spotify.com/v1/audio-analysis/57F8...,320547,4
8,1xz9uWsud2O07LXUcxN7vk,Crack the Tomb,Hulvey,COMA,6,2022-09-24 07:08:15,"[christian hip hop, christian trap]",2021,63,0.750,0.569,11,-4.929,0,0.2030,0.103000,0.000000,0.3290,0.470,86.530,audio_features,spotify:track:1xz9uWsud2O07LXUcxN7vk,https://api.spotify.com/v1/tracks/1xz9uWsud2O0...,https://api.spotify.com/v1/audio-analysis/1xz9...,134578,4
9,1jDJFeK9x3OZboIAHsY9k2,I'm Still Standing,Elton John,Too Low For Zero,6,2022-09-24 07:05:56,"[glam rock, mellow gold, piano rock]",1983,85,0.504,0.904,6,-6.863,1,0.1790,0.356000,0.121000,0.1400,0.772,176.808,audio_features,spotify:track:1jDJFeK9x3OZboIAHsY9k2,https://api.spotify.com/v1/tracks/1jDJFeK9x3OZ...,https://api.spotify.com/v1/audio-analysis/1jDJ...,183440,4


# Finding Final Playlist Vector

In [25]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added', 'month_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    playlist_feature_set['weight'] = playlist_feature_set['month_added'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [26]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, user_df, 1.20)
complete_feature_set_nonplaylist

,genre|acoustic,genre|adult,genre|afrofuturism,genre|afropop,genre|afroswing,genre|alabama,genre|album,genre|alt,genre|alternative,genre|ambient,genre|and,genre|anime,genre|anthem,genre|antigas,genre|arena,genre|argentine,genre|argentino,genre|art,genre|athens,genre|atl,genre|austindie,genre|australian,genre|azontobeats,genre|bachata,genre|background,genre|baiano,genre|baixada,genre|band,genre|banda,genre|barbadian,genre|baroque,genre|bass,genre|basshall,genre|baton,genre|beatlesque,genre|beats,genre|bedroom,genre|belarusian,genre|belgian,genre|bh,genre|big,genre|birmingham,genre|blues,genre|bmore,genre|bollywood,genre|boricua,genre|boston,genre|bounce,genre|boy,genre|brasileira,genre|brasileiro,genre|brazilian,genre|brill,genre|british,genre|britpop,genre|broadway,genre|brooklyn,genre|brostep,genre|bubblegrunge,genre|bubblegum,genre|building,genre|cafe,genre|cali,genre|canadian,genre|candy,genre|carioca,genre|carolina,genre|ccm,genre|celtic,genre|champeta,genre|channel,genre|chicago,genre|chicano,genre|chihuahua,genre|chileno,genre|chill,genre|chillhop,genre|chillwave,genre|christian,genre|circuit,genre|classic,genre|cloud,genre|coast,genre|colombian,genre|colombiana,genre|colombiano,genre|conscient,genre|consciente,genre|conscious,genre|contemporary,genre|corrido,genre|corridos,genre|country,genre|cristiano,genre|cubaton,genre|cumbia,genre|czech,genre|czsk,genre|dance,genre|dancehall,genre|dark,genre|das,genre|dawn,genre|de,genre|deep,genre|dembow,genre|desi,genre|detroit,genre|df,genre|dfw,genre|diego,genre|dirty,genre|disco,genre|dmv,genre|dominicana,genre|dominicano,genre|double,genre|dream,genre|drill,genre|drumming,genre|drums,genre|dubstep,genre|dutch,genre|east,genre|edm,genre|edmonton,genre|el,genre|electro,genre|electronic,genre|electropop,genre|emo,genre|en,genre|escape,genre|espanol,genre|etherpop,genre|euro,genre|eurodance,genre|europop,genre|experimental,genre|favela,genre|fi,genre|filmi,genre|filter,genre|flamenco,genre|florida,genre|flow,genre|fluminense,genre|folk,genre|forro,genre|francais,genre|francoton,genre|french,genre|funk,genre|funky,genre|fusion,genre|game,genre|gaming,genre|gangster,genre|garage,genre|gauze,genre|german,genre|girl,genre|glam,genre|gold,genre|greek,genre|groove,genre|group,genre|grunge,genre|guaracha,genre|hamburg,genre|hard,genre|hardcore,genre|hawaiian,genre|heartland,genre|hi,genre|hip,genre|holler,genre|hollywood,genre|hop,genre|house,genre|houston,genre|icelandic,genre|idol,genre|indie,genre|indiecoustica,genre|indietronica,genre|indonesian,genre|international,genre|invasion,genre|irish,genre|island,genre|islands,genre|italian,genre|italiano,genre|italo,genre|japanese,genre|jersey,genre|kentucky,genre|kombina,genre|la,genre|latin,genre|latina,genre|latino,genre|lgbtq,genre|lilith,genre|lo,genre|lovers,genre|madchester,genre|mariachi,genre|marseille,genre|media,genre|melancholia,genre|melbourne,genre|mellow,genre|melodic,genre|meme,genre|merseybeat,genre|metal,genre|metalcore,genre|metropopolis,genre|mexican,genre|mexicano,genre|miami,genre|milan,genre|minneapolis,genre|minnesota,genre|mississippi,genre|mn,genre|modern,genre|moombahton,genre|motown,genre|movie,genre|mpb,genre|music,genre|musica,genre|nacional,genre|neo,genre|neon,genre|new,genre|newcastle,genre|nigerian,genre|ninja,genre|noise,genre|norteno,genre|north,genre|norwegian,genre|nova,genre|novo,genre|nrg,genre|nsw,genre|nu,genre|nuevo,genre|nyc,genre|nz,genre|oakland,genre|ohio,genre|oklahoma,genre|old,genre|orleans,genre|ostentacao,genre|otacore,genre|outlaw,genre|oxford,genre|pagode,genre|panamanian,genre|paso,genre|paulista,genre|permanent,genre|peruvian,genre|philly,genre|piano,genre|piseiro,genre|pittsburgh,genre|pixie,genre|png,genre|polish,genre|political,genre|polynesian,genre|pop,genre|poprock,genre|poptimism,genre|popular,genre|portland,genre|portuguese,genre|post,genre|power,genre|progressive,genre|psych,genre|psychedelic,genre|puerto,genre|punjabi,genre|punk,genre|queen,genre|queens,genre|queer,genre|quiet

# Generate recommendations

In [27]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [ ]:
spotify_df.rename(columns={'spotify_id': 'id'}, inplace=True)
recommendation = generate_playlist_recos(spotify_df, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
recommendation

# Upload as album to Spotify 

In [29]:
# Create a playlist for user
results_playlist = sp.user_playlist_create(userid, "recommendation", public=False, collaborative=False, description='test')
playlist_id = results_playlist['id']

In [30]:
# Adding songs to playlist
songs = list(recommendation['id'])
sp.playlist_add_items(playlist_id, songs, position=None)

{'snapshot_id': 'MyxiM2I1NGQ4ZjkxNTEzNWVmNmUyYjUxYWE3OWU5NTUxYjk1Y2ZlYzIx'}